**Output Parsers**

    - LLM의 출력을 받아 더 적합한 형식으로 변환하는 역할
    - LLM을 사용해서 어떤 형태의 구조화된 데이터를 생성할 때 매우 유용


**PydanticOutputParser**

    - 언어 모델의 출력을 구조화된 정보로 변환하는데 도움이 되는 클래스
    - 언어 모델에서 나오는 단순 텍스트를 명확하고 체계적인 형태로 제공함
    - 핵심 메서드는 `get_format_instructions()` : 언어 모델이 출력해야 할 정보의 형식을 정의하는 지침 제공
    언어 모델이 출력해야 할 데이터의 필드와 그 형태를 설명하는 지침을 문자열로 반환. 사용자가 원하는 출력 형식을 정확히 이해하고 준비하는 데 도움을 줌
    `parse()` : 언어 모델의 출력(문자열로 가정)을 받아들여 특정 구조로 분석하고 변환
    Pydantic과 같은 도구를 사용해 입력된 문자열을 사전 정의된 스키마에 따라 검증하고 해당 스키마를 따르는 데이터 구조로 변환


In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.output_parsers import PydanticOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field

In [3]:
llm = ChatOpenAI(
    model='gpt-3.5-turbo',
    temperature=0,
)

In [4]:
email_conversation = """From: 김철수 (chulsoo.kim@bikecorporation.me)
To: 이은채 (eunchae@teddyinternational.me)
Subject: "ZENESIS" 자전거 유통 협력 및 미팅 일정 제안

안녕하세요, 이은채 대리님,

저는 바이크코퍼레이션의 김철수 상무입니다. 최근 보도자료를 통해 귀사의 신규 자전거 "ZENESIS"에 대해 알게 되었습니다. 바이크코퍼레이션은 자전거 제조 및 유통 분야에서 혁신과 품질을 선도하는 기업으로, 이 분야에서의 장기적인 경험과 전문성을 가지고 있습니다.

ZENESIS 모델에 대한 상세한 브로슈어를 요청드립니다. 특히 기술 사양, 배터리 성능, 그리고 디자인 측면에 대한 정보가 필요합니다. 이를 통해 저희가 제안할 유통 전략과 마케팅 계획을 보다 구체화할 수 있을 것입니다.

또한, 협력 가능성을 더 깊이 논의하기 위해 다음 주 화요일(1월 15일) 오전 10시에 미팅을 제안합니다. 귀사 사무실에서 만나 이야기를 나눌 수 있을까요?

감사합니다.

김철수
상무이사
바이크코퍼레이션
"""


In [5]:
class EmailSummary(BaseModel):
    person : str = Field(description='메일을 보낸 사람')
    email : str = Field(description='메일을 보낸 사람의 이메일 주소')
    subject : str = Field(description='메일 제목')
    summary : str = Field(description='메일 본문을 요약한 텍스트')
    date : str = Field(description = '메일 본문에 언급된 미팅 날짜와 시간')
    
# PydanticOutputParser 생성
parser = PydanticOutputParser(pydantic_object=EmailSummary)
parser

PydanticOutputParser(pydantic_object=<class '__main__.EmailSummary'>)

프롬프트 정의

     - question : 유저의 질문
     - email_conversation : 이메일 본문 내용 입력
     - format : 형식

In [6]:
prompt = PromptTemplate.from_template(
    """
You are a helpful assistant. Please answer the following question is KOREAN.

QUESTION:
{question}

EMAIL_CONVERSATION:
{email_conversation}

FORMAT:
{format}
"""
)

#format에 PydanticOutputParser의 format을 추가
prompt = prompt.partial(format=parser.get_format_instructions())
prompt

PromptTemplate(input_variables=['email_conversation', 'question'], partial_variables={'format': 'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"person": {"title": "Person", "description": "\\uba54\\uc77c\\uc744 \\ubcf4\\ub0b8 \\uc0ac\\ub78c", "type": "string"}, "email": {"title": "Email", "description": "\\uba54\\uc77c\\uc744 \\ubcf4\\ub0b8 \\uc0ac\\ub78c\\uc758 \\uc774\\uba54\\uc77c \\uc8fc\\uc18c", "type": "string"}, "subject": {"title": "Subject", "description": "\\uba54\\uc77c \\uc81c\\ubaa9", "type": "string"}, "summary": {"title": "Summary", "description": "\\

In [7]:
chain = prompt | llm
chain

PromptTemplate(input_variables=['email_conversation', 'question'], partial_variables={'format': 'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"person": {"title": "Person", "description": "\\uba54\\uc77c\\uc744 \\ubcf4\\ub0b8 \\uc0ac\\ub78c", "type": "string"}, "email": {"title": "Email", "description": "\\uba54\\uc77c\\uc744 \\ubcf4\\ub0b8 \\uc0ac\\ub78c\\uc758 \\uc774\\uba54\\uc77c \\uc8fc\\uc18c", "type": "string"}, "subject": {"title": "Subject", "description": "\\uba54\\uc77c \\uc81c\\ubaa9", "type": "string"}, "summary": {"title": "Summary", "description": "\\

In [8]:
response = chain.invoke(
    {
        'email_conversation' : email_conversation,
        'question' : '이메일 내용 중 주요 내용을 추출해 주세요.'
    }
)

print(response.content)

{
  "person": "김철수",
  "email": "chulsoo.kim@bikecorporation.me",
  "subject": "ZENESIS 자전거 유통 협력 및 미팅 일정 제안",
  "summary": "ZENESIS 모델에 대한 상세한 브로슈어를 요청하고, 협력 가능성을 논의하기 위해 미팅 일정을 제안함",
  "date": "다음 주 화요일(1월 15일) 오전 10시"
}


In [9]:
parser.parse(response.content)

EmailSummary(person='김철수', email='chulsoo.kim@bikecorporation.me', subject='ZENESIS 자전거 유통 협력 및 미팅 일정 제안', summary='ZENESIS 모델에 대한 상세한 브로슈어를 요청하고, 협력 가능성을 논의하기 위해 미팅 일정을 제안함', date='다음 주 화요일(1월 15일) 오전 10시')

**parser가 결합된 체인 생성**

In [14]:
chain = prompt | llm | parser

In [16]:
response = chain.invoke(
    {
        "email_conversation": email_conversation,
        "question": "이메일 내용 추출해줘",
    }
)

# 결과는 EmailSummary 객체 형태로 출력
response


OutputParserException: Failed to parse EmailSummary from completion {'properties': {'person': {'title': 'Person', 'description': '이메일을 발신한 사람', 'type': 'string'}, 'email': {'title': 'Email', 'description': '이메일을 발신한 사람의 이메일 주소', 'type': 'string'}, 'subject': {'title': 'Subject', 'description': '이메일 제목', 'type': 'string'}, 'summary': {'title': 'Summary', 'description': '이메일 내용을 요약한 문자열', 'type': 'string'}, 'date': {'title': 'Date', 'description': '이메일 발신일과 시간', 'type': 'string'}}, 'required': ['person', 'email', 'subject', 'summary', 'date']}. Got: 5 validation errors for EmailSummary
person
  field required (type=value_error.missing)
email
  field required (type=value_error.missing)
subject
  field required (type=value_error.missing)
summary
  field required (type=value_error.missing)
date
  field required (type=value_error.missing)